# Purpose
This notebook runs the `vectorize_text_to_embeddings` function to:
- loading fastText embeddings & create a uSIF model
- load post & comment text
- train a uSIF model
- convert the text into embeddings (at post or comment level)

Currently only one job call runs at a time, so I may try running two notebooks at the same time to run some jobs in parallel.


# Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from datetime import datetime
import gc
from functools import partial
import os
import logging
from pathlib import Path
from pprint import pprint

import mlflow

import numpy as np
import pandas as pd

from subclu.models.vectorize_text import (
    vectorize_text_to_embeddings,
    D_MODELS_CPU,
    process_text_for_fse,
    vectorize_text_with_fse,
)
from subclu.models.preprocess_text import TextPreprocessor, transform_and_tokenize_text

from subclu.utils import set_working_directory
from subclu.utils.mlflow_logger import MlflowLogger
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric
)


print_lib_versions([mlflow, np, mlflow, pd])

python		v 3.7.10
===
mlflow		v: 1.16.0
numpy		v: 1.19.5
mlflow		v: 1.16.0
pandas		v: 1.2.4


In [5]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Set sqlite database as MLflow URI

In [6]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')

In [7]:
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/mlflow/mlruns.db'

# Get list of experiments

### with new function

In [10]:
logging.getLogger('sqlalchemy.engine').setLevel(logging.WARN)

In [18]:
mlf.list_experiment_meta()

[{'experiment_id': '0',
  'name': 'Default',
  'artifact_location': './mlruns/0',
  'lifecycle_stage': 'active'},
 {'experiment_id': '1',
  'name': 'fse_v1',
  'artifact_location': 'gs://i18n-subreddit-clustering/mlflow/mlruns/1',
  'lifecycle_stage': 'active'},
 {'experiment_id': '2',
  'name': 'fse_vectorize_v1',
  'artifact_location': 'gs://i18n-subreddit-clustering/mlflow/mlruns/2',
  'lifecycle_stage': 'active'},
 {'experiment_id': '3',
  'name': 'subreddit_description_v1',
  'artifact_location': 'gs://i18n-subreddit-clustering/mlflow/mlruns/3',
  'lifecycle_stage': 'active'}]

# Call function to vectorize text

In [20]:
mlflow_experiment = 'fse_vectorize_v1.1'

In [21]:
# del model, df_posts, d_ix_to_id
gc.collect()

mlflow.end_run(status='KILLED')
model, df_posts, d_ix_to_id = vectorize_text_to_embeddings(
    mlflow_experiment=mlflow_experiment,
    
    tokenize_function='sklearn_acronyms_emoji',
    tokenize_lowercase=True,
    train_min_word_count=4,
    train_subreddits_to_exclude=['wixbros', 'katjakrasavicenudes',
                                 'deutschetributes', 'germannudes'],
)

07:08:44 | INFO | "Start vectorize function"
07:08:44 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/fse/2021-06-02_0708"
07:08:44 | INFO | "Loading df_posts...
  gs://i18n-subreddit-clustering/posts/2021-05-19"
07:08:53 | INFO | "  0:00:09.099387 <- df_post time elapsed"
07:08:53 | INFO | "  (111669, 6) <- df_posts.shape"
07:08:53 | INFO | "Load comments df..."
07:09:05 | INFO | "  (757388, 6) <- df_comments shape"
07:09:06 | INFO | "Keep only comments that match posts IDs in df_posts..."
07:09:06 | INFO | "  (638052, 6) <- updated df_comments shape"
07:09:06 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/mlflow/mlruns.db"
07:09:06 | INFO | "Filtering posts for SIF training..."
07:09:07 | INFO | "38,296 <- Exclude posts because of: subreddits filter"
07:09:07 | INFO | "30,537 <- Exclude posts because of: duplicated posts"
07:09:07 | INFO | "25,328 <- Exclude posts because of: minimum word count"
07:09:07 | INFO | "43,452 <- df_pos

In [22]:
mlflow.end_run(status='KILLED')

model, df_posts, d_ix_to_id = vectorize_text_to_embeddings(
    mlflow_experiment=mlflow_experiment,
    
    tokenize_function='sklearn',
    tokenize_lowercase=True,
    train_min_word_count=4,
)

07:12:14 | INFO | "Start vectorize function"
07:12:14 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/fse/2021-06-02_0712"
07:12:14 | INFO | "Loading df_posts...
  gs://i18n-subreddit-clustering/posts/2021-05-19"
07:12:21 | INFO | "  0:00:07.399874 <- df_post time elapsed"
07:12:21 | INFO | "  (111669, 6) <- df_posts.shape"
07:12:21 | INFO | "Load comments df..."
07:12:31 | INFO | "  (757388, 6) <- df_comments shape"
07:12:31 | INFO | "Keep only comments that match posts IDs in df_posts..."
07:12:31 | INFO | "  (638052, 6) <- updated df_comments shape"
07:12:31 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/mlflow/mlruns.db"
07:12:35 | INFO | "Filtering posts for SIF training..."
07:12:35 | INFO | "     0 <- Exclude posts because of: subreddits filter"
07:12:35 | INFO | "30,537 <- Exclude posts because of: duplicated posts"
07:12:35 | INFO | "25,328 <- Exclude posts because of: minimum word count"
07:12:35 | INFO | "69,122 <- df_pos

In [23]:
del model, df_posts, d_ix_to_id
gc.collect()

mlflow.end_run(status='KILLED')
model, df_posts, d_ix_to_id = vectorize_text_to_embeddings(
    mlflow_experiment=mlflow_experiment,
    
    tokenize_function='sklearn_emoji',
    tokenize_lowercase=True,
    train_min_word_count=4,
    train_subreddits_to_exclude=['wixbros', 'katjakrasavicenudes',
                                 'deutschetributes', 'germannudes'],
)

07:15:29 | INFO | "Start vectorize function"
07:15:29 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/fse/2021-06-02_0715"
07:15:29 | INFO | "Loading df_posts...
  gs://i18n-subreddit-clustering/posts/2021-05-19"
07:15:35 | INFO | "  0:00:06.756756 <- df_post time elapsed"
07:15:35 | INFO | "  (111669, 6) <- df_posts.shape"
07:15:35 | INFO | "Load comments df..."
07:15:44 | INFO | "  (757388, 6) <- df_comments shape"
07:15:45 | INFO | "Keep only comments that match posts IDs in df_posts..."
07:15:45 | INFO | "  (638052, 6) <- updated df_comments shape"
07:15:45 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/mlflow/mlruns.db"
07:15:45 | INFO | "Filtering posts for SIF training..."
07:15:45 | INFO | "38,296 <- Exclude posts because of: subreddits filter"
07:15:45 | INFO | "30,537 <- Exclude posts because of: duplicated posts"
07:15:45 | INFO | "25,328 <- Exclude posts because of: minimum word count"
07:15:45 | INFO | "43,452 <- df_pos

In [30]:
try:
    del model, df_posts, d_ix_to_id
except NameError:
    pass
gc.collect()

mlflow.end_run(status='KILLED')
model, df_posts, d_ix_to_id = vectorize_text_to_embeddings(
    mlflow_experiment=mlflow_experiment,
    
    tokenize_function='gensim',
    tokenize_lowercase=True,
    train_min_word_count=4,
    train_subreddits_to_exclude=['wixbros', 'katjakrasavicenudes',
                                 'deutschetributes', 'germannudes'],
)

07:46:02 | INFO | "Start vectorize function"
07:46:02 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/fse/2021-06-02_0746"
07:46:02 | INFO | "Loading df_posts...
  gs://i18n-subreddit-clustering/posts/2021-05-19"
07:46:07 | INFO | "  0:00:04.587122 <- df_post time elapsed"
07:46:07 | INFO | "  (111669, 6) <- df_posts.shape"
07:46:07 | INFO | "Load comments df..."
07:46:13 | INFO | "  (757388, 6) <- df_comments shape"
07:46:14 | INFO | "Keep only comments that match posts IDs in df_posts..."
07:46:14 | INFO | "  (638052, 6) <- updated df_comments shape"
07:46:14 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/mlflow/mlruns.db"
07:46:14 | INFO | "Filtering posts for SIF training..."
07:46:14 | INFO | "38,296 <- Exclude posts because of: subreddits filter"
07:46:14 | INFO | "30,537 <- Exclude posts because of: duplicated posts"
07:46:14 | INFO | "25,328 <- Exclude posts because of: minimum word count"
07:46:14 | INFO | "43,452 <- df_pos

In [31]:
try:
    del model, df_posts, d_ix_to_id
except NameError:
    pass
gc.collect()

mlflow.end_run(status='KILLED')
model, df_posts, d_ix_to_id = vectorize_text_to_embeddings(
    mlflow_experiment=mlflow_experiment,
    
    tokenize_function='gensim',
    tokenize_lowercase=False,
    train_min_word_count=4,
    train_subreddits_to_exclude=['wixbros', 'katjakrasavicenudes',
                                 'deutschetributes', 'germannudes'],
)

07:50:00 | INFO | "Start vectorize function"
07:50:00 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/fse/2021-06-02_0750"
07:50:00 | INFO | "Loading df_posts...
  gs://i18n-subreddit-clustering/posts/2021-05-19"
07:50:05 | INFO | "  0:00:04.530810 <- df_post time elapsed"
07:50:05 | INFO | "  (111669, 6) <- df_posts.shape"
07:50:05 | INFO | "Load comments df..."
07:50:12 | INFO | "  (757388, 6) <- df_comments shape"
07:50:12 | INFO | "Keep only comments that match posts IDs in df_posts..."
07:50:12 | INFO | "  (638052, 6) <- updated df_comments shape"
07:50:12 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/mlflow/mlruns.db"
07:50:13 | INFO | "Filtering posts for SIF training..."
07:50:13 | INFO | "38,296 <- Exclude posts because of: subreddits filter"
07:50:13 | INFO | "30,537 <- Exclude posts because of: duplicated posts"
07:50:13 | INFO | "25,328 <- Exclude posts because of: minimum word count"
07:50:13 | INFO | "43,452 <- df_pos

# Recover artifact from mlflow

In [24]:
run_id = '885f983de7e04022a8327cb16afd2c4f'
run = mlflow.get_run(run_id)

In [25]:
f"{run.info.artifact_uri}/d_ix_to_id/d_ix_to_id.csv"

'gs://i18n-subreddit-clustering/mlflow/mlruns/2/885f983de7e04022a8327cb16afd2c4f/artifacts/d_ix_to_id/d_ix_to_id.csv'

In [26]:
df_idx = pd.read_csv(f"{run.info.artifact_uri}/d_ix_to_id/d_ix_to_id.csv")
df_idx.head()

,training_index,post_id
0,0,t3_mkyj2k
1,1,t3_mkynzi
2,2,t3_mkyolv
3,3,t3_mkyp17
4,4,t3_mkyqrz
